In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from  sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.metrics import accuracy_score,confusion_matrix

In [16]:
# import os
# path='C:/Users/hp/Urban-Air-Quality-Monitoring/'
# extension='.csv'

In [17]:
# files = [file for file in os.listdir(path) if file.endswith(extension)]

In [18]:
# dfs=[]
# for file in files:
#     df=pd.read_csv(os.path.join(path,file))
#     dfs.append(df)

In [19]:
# df = pd.concat(dfs, ignore_index=True)
df=pd.read_csv("city_day.csv")
missing_values = df['AQI'].isnull()
df.dropna(subset=['AQI'], inplace=True)
missing_values.value_counts()
df.head()

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
28,Ahmedabad,2015-01-29,83.13,NaN,NaN,28.71,33.72,NaN,6.93,49.52,59.76,0.02,0.00,3.14,209.0,Poor
29,Ahmedabad,2015-01-30,79.84,NaN,NaN,28.68,41.08,NaN,13.85,48.49,97.07,0.04,0.00,4.81,328.0,Very Poor
30,Ahmedabad,2015-01-31,94.52,NaN,NaN,32.66,52.61,NaN,24.39,67.39,111.33,0.24,0.01,7.67,514.0,Severe
31,Ahmedabad,2015-02-01,135.99,NaN,NaN,42.08,84.57,NaN,43.48,75.23,102.70,0.40,0.04,25.87,782.0,Severe
32,Ahmedabad,2015-02-02,178.33,NaN,NaN,35.31,72.80,NaN,54.56,55.04,107.38,0.46,0.06,35.61,914.0,Severe


In [20]:
df = df.drop(columns=['City'])

In [21]:
df = df.drop(columns=['Date'])

In [22]:
df

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
28,83.13,NaN,NaN,28.71,33.72,NaN,6.93,49.52,59.76,0.02,0.00,3.14,209.0,Poor
29,79.84,NaN,NaN,28.68,41.08,NaN,13.85,48.49,97.07,0.04,0.00,4.81,328.0,Very Poor
30,94.52,NaN,NaN,32.66,52.61,NaN,24.39,67.39,111.33,0.24,0.01,7.67,514.0,Severe
31,135.99,NaN,NaN,42.08,84.57,NaN,43.48,75.23,102.70,0.40,0.04,25.87,782.0,Severe
32,178.33,NaN,NaN,35.31,72.80,NaN,54.56,55.04,107.38,0.46,0.06,35.61,914.0,Severe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25550,16.88,31.28,4.32,10.18,13.71,5.55,0.32,4.24,30.17,NaN,NaN,NaN,59.0,Satisfactory
25551,18.57,35.34,5.70,9.56,12.70,4.97,0.46,6.00,34.58,NaN,NaN,NaN,53.0,Satisfactory
25552,18.85,37.62,5.44,8.86,12.09,4.39,0.47,5.79,51.56,NaN,NaN,NaN,61.0,Satisfactory
25553,19.06,45.18,3.37,7.91,9.92,4.37,0.52,5.76,46.01,NaN,NaN,NaN,63.0,Satisfactory


In [23]:
nullvalues=df.isnull().sum().sort_values(ascending=False)

In [24]:
null_values_percentage=(df.isnull().sum()/df.isnull().count() *100).sort_values(ascending=False)

In [25]:
missing_null_values_with_percentage=pd.concat([nullvalues,null_values_percentage],axis=1,keys=['Total','Percentage'])
missing_null_values_with_percentage

,Total,Percentage
Xylene,13842,64.973714
PM10,6931,32.533796
NH3,6282,29.487420
Toluene,5289,24.826324
Benzene,3107,14.584116
NOx,1762,8.270747
NO,1533,7.195832
PM2.5,621,2.914946
O3,586,2.750657
SO2,528,2.478408


In [26]:
df.fillna(0,inplace=True)

In [27]:
df.isnull().sum()

PM2.5         0
PM10          0
NO            0
NO2           0
NOx           0
NH3           0
CO            0
SO2           0
O3            0
Benzene       0
Toluene       0
Xylene        0
AQI           0
AQI_Bucket    0
dtype: int64

In [28]:
def calculate_si(so2):
    si=0
    if (so2<=40):
     si= so2*(50/40)
    if (so2>40 and so2<=80):
     si= 50+(so2-40)*(50/40)
    if (so2>80 and so2<=380):
     si= 100+(so2-80)*(100/300)
    if (so2>380 and so2<=800):
     si= 200+(so2-380)*(100/800)
    if (so2>800 and so2<=1600):
     si= 300+(so2-800)*(100/800)
    if (so2>1600):
     si= 400+(so2-1600)*(100/800)
    return si
df['si']=df['SO2'].apply(calculate_si)
data= df[['SO2','si']]
data.head()

,SO2,si
28,49.52,61.9000
29,48.49,60.6125
30,67.39,84.2375
31,75.23,94.0375
32,55.04,68.8000


In [29]:
def calculate_no2_si(no2):
    si = 0
    if no2 <= 40:
        si = no2 * (50 / 40)
    elif no2 <= 80:
        si = 50 + (no2 - 40) * (50 / 40)
    elif no2 <= 180:
        si = 100 + (no2 - 80) * (100 / 100)
    elif no2 <= 280:
        si = 200 + (no2 - 180) * (100 / 100)
    elif no2 <= 400:
        si = 300 + (no2 - 280) * (100 / 120)
    else:
        si = 400 + (no2 - 400) * (100 / 120)
    return si
df['no2i']=df['NO2'].apply(calculate_si)
data= df[['NO2','no2i']]
data.head()

,NO2,no2i
28,28.71,35.8875
29,28.68,35.8500
30,32.66,40.8250
31,42.08,52.6000
32,35.31,44.1375


In [30]:
def calculate_o3_si(o3):
    si = 0
    if o3 <= 50:
        si = o3 * (50 / 50)
    elif o3 <= 100:
        si = 50 + (o3 - 50) * (50 / 50)
    elif o3 <= 168:
        si = 100 + (o3 - 100) * (100 / 68)
    elif o3 <= 208:
        si = 200 + (o3 - 168) * (100 / 40)
    elif o3 <= 748:
        si = 300 + (o3 - 208) * (100 / 540)
    else:
        si = 400 + (o3 - 748) * (100 / 540)
    return si
df['o3i']=df['O3'].apply(calculate_si)
data= df[['O3','o3i']]
data.head()

,O3,o3i
28,59.76,74.700000
29,97.07,105.690000
30,111.33,110.443333
31,102.70,107.566667
32,107.38,109.126667


In [31]:
def calculate_pm10_si(pm10):
    si = 0
    if pm10 <= 50:
        si = pm10 * (50 / 50)
    elif pm10 <= 100:
        si = 50 + (pm10 - 50) * (50 / 50)
    elif pm10 <= 250:
        si = 100 + (pm10 - 100) * (100 / 150)
    elif pm10 <= 350:
        si = 200 + (pm10 - 250) * (100 / 100)
    elif pm10 <= 430:
        si = 300 + (pm10 - 350) * (100 / 80)
    else:
        si = 400 + (pm10 - 430) * (100 / 80)
    return si
df['pm10i']=df['PM10'].apply(calculate_si)
data= df[['PM10','pm10i']]
data.head()


,PM10,pm10i
28,0.0,0.0
29,0.0,0.0
30,0.0,0.0
31,0.0,0.0
32,0.0,0.0


In [32]:
def calculate_no_si(no):
    si = 0
    if no <= 40:
        si = no * (50 / 40)
    elif no <= 80:
        si = 50 + (no - 40) * (50 / 40)
    elif no <= 180:
        si = 100 + (no - 80) * (100 / 100)
    elif no <= 280:
        si = 200 + (no - 180) * (100 / 100)
    elif no <= 400:
        si = 300 + (no - 280) * (100 / 120)
    else:
        si = 400 + (no - 400) * (100 / 120)
    return si
df['noi']=df['NO'].apply(calculate_si)
data= df[['NO','noi']]
data.head()

,NO,noi
28,0.0,0.0
29,0.0,0.0
30,0.0,0.0
31,0.0,0.0
32,0.0,0.0


In [33]:
def calculate_nh3_si(nh3):
    si = 0
    if nh3 <= 200:
        si = nh3 * (50 / 200)
    elif nh3 <= 400:
        si = 50 + (nh3 - 200) * (50 / 200)
    elif nh3 <= 800:
        si = 100 + (nh3 - 400) * (100 / 400)
    elif nh3 <= 1200:
        si = 200 + (nh3 - 800) * (100 / 400)
    elif nh3 <= 1800:
        si = 300 + (nh3 - 1200) * (100 / 600)
    else:
        si = 400 + (nh3 - 1800) * (100 / 600)
    return si
df['nh3i']=df['NH3'].apply(calculate_si)
data= df[['NH3','nh3i']]
data.head()

,NH3,nh3i
28,0.0,0.0
29,0.0,0.0
30,0.0,0.0
31,0.0,0.0
32,0.0,0.0


In [34]:
def calculate_nox_si(nox):
    si = 0
    if nox <= 40:
        si = nox * (50 / 40)
    elif 40 < nox <= 80:
        si = 50 + (nox - 40) * (50 / 40)
    elif 80 < nox <= 180:
        si = 100 + (nox - 80) * (100 / 100)
    elif 180 < nox <= 280:
        si = 200 + (nox - 180) * (100 / 100)
    elif 280 < nox <= 400:
        si = 300 + (nox - 280) * (100 / 120)
    elif nox > 400:
        si = 400 + (nox - 400) * (100 / 400)
    return si
df['noxi']=df['NOx'].apply(calculate_si)
data= df[['NOx','noxi']]
data.head()

,NOx,noxi
28,33.72,42.150000
29,41.08,51.350000
30,52.61,65.762500
31,84.57,101.523333
32,72.80,91.000000


In [35]:
def calculate_co_si(co):
    si = 0
    if co <= 1:
        si = co * (50 / 1)
    elif 1 < co <= 2:
        si = 50 + (co - 1) * (50 / 1)
    elif 2 < co <= 10:
        si = 100 + (co - 2) * (100 / 8)
    elif 10 < co <= 17:
        si = 200 + (co - 10) * (100 / 7)
    elif 17 < co <= 34:
        si = 300 + (co - 17) * (100 / 17)
    elif co > 34:
        si = 400 + (co - 34) * (100 / 66)
    return si

def calculate_o3_si(o3):
    si = 0
    if o3 <= 50:
        si = o3 * (50 / 50)
    elif 50 < o3 <= 100:
        si = 50 + (o3 - 50) * (50 / 50)
    elif 100 < o3 <= 168:
        si = 100 + (o3 - 100) * (100 / 68)
    elif 168 < o3 <= 208:
        si = 200 + (o3 - 168) * (100 / 40)
    elif 208 < o3 <= 748:
        si = 300 + (o3 - 208) * (100 / 539)
    elif o3 > 748:
        si = 400 + (o3 - 748) * (100 / 254)
    return si

def calculate_benzene_si(benzene):
    si = 0
    if benzene <= 5:
        si = benzene * (50 / 5)
    elif 5 < benzene <= 10:
        si = 50 + (benzene - 5) * (50 / 5)
    elif 10 < benzene <= 15:
        si = 100 + (benzene - 10) * (100 / 5)
    elif 15 < benzene <= 20:
        si = 200 + (benzene - 15) * (100 / 5)
    elif 20 < benzene <= 25:
        si = 300 + (benzene - 20) * (100 / 5)
    elif benzene > 25:
        si = 400 + (benzene - 25) * (100 / 25)
    return si

def calculate_toluene_si(toluene):
    si = 0
    if toluene <= 5:
        si = toluene * (50 / 5)
    elif 5 < toluene <= 10:
        si = 50 + (toluene - 5) * (50 / 5)
    elif 10 < toluene <= 20:
        si = 100 + (toluene - 10) * (100 / 10)
    elif 20 < toluene <= 30:
        si = 200 + (toluene - 20) * (100 / 10)
    elif 30 < toluene <= 40:
        si = 300 + (toluene - 30) * (100 / 10)
    elif toluene > 40:
        si = 400 + (toluene - 40) * (100 / 10)
    return si

def calculate_xylene_si(xylene):
    si = 0
    if xylene <= 5:
        si = xylene * (50 / 5)
    elif 5 < xylene <= 10:
        si = 50 + (xylene - 5) * (50 / 5)
    elif 10 < xylene <= 15:
        si = 100 + (xylene - 10) * (100 / 5)
    elif 15 < xylene <= 20:
        si = 200 + (xylene - 15) * (100 / 5)
    elif 20 < xylene <= 25:
        si = 300 + (xylene - 20) * (100 / 5)
    elif xylene > 25:
        si = 400 + (xylene - 25) * (100 / 25)
    return si

df['xylenei']=df['Xylene'].apply(calculate_si)
data= df[['Xylene','xylenei']]
data.head()

,Xylene,xylenei
28,3.14,3.9250
29,4.81,6.0125
30,7.67,9.5875
31,25.87,32.3375
32,35.61,44.5125


In [36]:
df['toluenei']=df['Toluene'].apply(calculate_si)
data= df[['Toluene','toluenei']]
data.head()

,Toluene,toluenei
28,0.00,0.0000
29,0.00,0.0000
30,0.01,0.0125
31,0.04,0.0500
32,0.06,0.0750


In [37]:
df['benzenei']=df['Benzene'].apply(calculate_si)
data= df[['Benzene','benzenei']]
data.head()

,Benzene,benzenei
28,0.02,0.025
29,0.04,0.050
30,0.24,0.300
31,0.40,0.500
32,0.46,0.575


In [38]:
df['coi']=df['CO'].apply(calculate_si)
data= df[['CO','coi']]
data.head()

,CO,coi
28,6.93,8.6625
29,13.85,17.3125
30,24.39,30.4875
31,43.48,54.3500
32,54.56,68.2000


In [39]:
def calculate_pm25_si(pm25):
    si = 0
    if pm25 <= 30:
        si = pm25 * (50 / 30)
    elif pm25 > 30 and pm25 <= 60:
        si = 50 + (pm25 - 30) * (50 / 30)
    elif pm25 > 60 and pm25 <= 90:
        si = 100 + (pm25 - 60) * (100 / 30)
    elif pm25 > 90 and pm25 <= 120:
        si = 200 + (pm25 - 90) * (100 / 30)
    elif pm25 > 120 and pm25 <= 250:
        si = 300 + (pm25 - 120) * (100 / 130)
    elif pm25 > 250:
        si = 400 + (pm25 - 250) * (100 / 250)
    return si
df['pm25i']=df['PM2.5'].apply(calculate_pm25_si)
data= df[['PM2.5','pm25i']]
data.head()

,PM2.5,pm25i
28,83.13,177.100000
29,79.84,166.133333
30,94.52,215.066667
31,135.99,312.300000
32,178.33,344.869231


In [40]:
def AQI_Range(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.NaN
    
df['AQI_Bucket']=df['AQI'].apply(AQI_Range)
df

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,...,o3i,pm10i,noi,nh3i,noxi,xylenei,toluenei,benzenei,coi,pm25i
28,83.13,0.00,0.00,28.71,33.72,0.00,6.93,49.52,59.76,0.02,...,74.700000,0.000,0.0000,0.0000,42.150000,3.9250,0.0000,0.025,8.6625,177.100000
29,79.84,0.00,0.00,28.68,41.08,0.00,13.85,48.49,97.07,0.04,...,105.690000,0.000,0.0000,0.0000,51.350000,6.0125,0.0000,0.050,17.3125,166.133333
30,94.52,0.00,0.00,32.66,52.61,0.00,24.39,67.39,111.33,0.24,...,110.443333,0.000,0.0000,0.0000,65.762500,9.5875,0.0125,0.300,30.4875,215.066667
31,135.99,0.00,0.00,42.08,84.57,0.00,43.48,75.23,102.70,0.40,...,107.566667,0.000,0.0000,0.0000,101.523333,32.3375,0.0500,0.500,54.3500,312.300000
32,178.33,0.00,0.00,35.31,72.80,0.00,54.56,55.04,107.38,0.46,...,109.126667,0.000,0.0000,0.0000,91.000000,44.5125,0.0750,0.575,68.2000,344.869231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25550,16.88,31.28,4.32,10.18,13.71,5.55,0.32,4.24,30.17,0.00,...,37.712500,39.100,5.4000,6.9375,17.137500,0.0000,0.0000,0.000,0.4000,28.133333
25551,18.57,35.34,5.70,9.56,12.70,4.97,0.46,6.00,34.58,0.00,...,43.225000,44.175,7.1250,6.2125,15.875000,0.0000,0.0000,0.000,0.5750,30.950000
25552,18.85,37.62,5.44,8.86,12.09,4.39,0.47,5.79,51.56,0.00,...,64.450000,47.025,6.8000,5.4875,15.112500,0.0000,0.0000,0.000,0.5875,31.416667
25553,19.06,45.18,3.37,7.91,9.92,4.37,0.52,5.76,46.01,0.00,...,57.512500,56.475,4.2125,5.4625,12.400000,0.0000,0.0000,0.000,0.6500,31.766667


In [41]:
df['AQI_Bucket'].value_counts()

AQI_Bucket
Moderate        7502
Satisfactory    6570
Poor            2674
Very Poor       2308
Severe          1329
Good             921
Name: count, dtype: int64

Splitting Dataset into Independent and Dependent columns

In [42]:
X=df[['o3i','pm10i','noi','nh3i','noxi','xylenei','toluenei','benzenei','coi','pm25i','noxi','si','no2i']]
Y=df['AQI']
X.head()

,o3i,pm10i,noi,nh3i,noxi,xylenei,toluenei,benzenei,coi,pm25i,noxi,si,no2i
28,74.700000,0.0,0.0,0.0,42.150000,3.9250,0.0000,0.025,8.6625,177.100000,42.150000,61.9000,35.8875
29,105.690000,0.0,0.0,0.0,51.350000,6.0125,0.0000,0.050,17.3125,166.133333,51.350000,60.6125,35.8500
30,110.443333,0.0,0.0,0.0,65.762500,9.5875,0.0125,0.300,30.4875,215.066667,65.762500,84.2375,40.8250
31,107.566667,0.0,0.0,0.0,101.523333,32.3375,0.0500,0.500,54.3500,312.300000,101.523333,94.0375,52.6000
32,109.126667,0.0,0.0,0.0,91.000000,44.5125,0.0750,0.575,68.2000,344.869231,91.000000,68.8000,44.1375


In [43]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.5,random_state=70)
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(10652, 13) (10652, 13) (10652,) (10652,)


In [44]:
model=LinearRegression()
model.fit(X_train,Y_train)

LinearRegression()

In [45]:
#predicting train
train_pred=model.predict(X_train)
#predicting on test
test_pred=model.predict(X_test)

In [46]:
RMSE_train=(np.sqrt(metrics.mean_squared_error(Y_train,train_pred)))
RMSE_test=(np.sqrt(metrics.mean_squared_error(Y_test,test_pred)))
print(str(RMSE_train))
print(str(RMSE_test))
print('-'*50)
print(model.score(X_train,Y_train))
print(model.score(X_test,Y_test))

54.84041925543369
59.46862503261551
--------------------------------------------------
0.8539131021246629
0.8453060551704485


Decision Tree Regressor

In [47]:
DT=DecisionTreeRegressor()
DT.fit(X_train,Y_train)

DecisionTreeRegressor()

In [48]:
train_preds=DT.predict(X_train)
test_preds=DT.predict(X_test)

In [49]:
RMSE_train=(np.sqrt(metrics.mean_squared_error(Y_train,train_preds)))
RMSE_test=(np.sqrt(metrics.mean_squared_error(Y_test,test_preds)))
print(str(RMSE_train))
print(str(RMSE_test))
print('-'*50)
print(model.score(X_train,Y_train))
print(model.score(X_test,Y_test))

4.157918861025105
65.92066222618492
--------------------------------------------------
0.8539131021246629
0.8453060551704485


Random Forest Regressor

In [50]:
RF=RandomForestRegressor().fit(X_train.head(100000),Y_train.head(100000))

In [51]:
train_preds1=RF.predict(X_train)
test_preds1=RF.predict(X_test)

In [52]:
RMSE_train=(np.sqrt(metrics.mean_squared_error(Y_train,train_preds)))
RMSE_test=(np.sqrt(metrics.mean_squared_error(Y_test,test_preds)))
print(str(RMSE_train))
print(str(RMSE_test))
print('-'*50)
print(model.score(X_train,Y_train))
print(model.score(X_test,Y_test))

4.157918861025105
65.92066222618492
--------------------------------------------------
0.8539131021246629
0.8453060551704485


Classification Algorithms

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
